In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sympy import sieve
import time

In [2]:
nodes = pd.read_csv('./problem/cities.csv')
nodes = nodes.iloc[:,1:].values
n = len(nodes)
prime_ = sieve.primerange(0,n)
prime = list(prime_)

In [3]:
def score(solution,offset=0,raw=False):
    s = nodes[solution[:-1]]
    e = nodes[solution[1:]]
    out = distance(s,e)
    n = len(s)
    if raw:
        return out
    isten = (np.arange(n)+1+offset)%10==0
    notprime = ~np.isin(solution[:-1],prime)
    isextra = (isten&notprime)*0.1
    out += out*isextra
    return out

def distance(coor1,coor2):
    return np.sqrt(((coor1-coor2)**2).sum(axis=-1))


In [270]:
solution = pd.read_csv('./solution/base.csv').values[:,0]
score(solution).sum()

1516912.3751145517

In [271]:
def calc_reverse(left, solution):
    r_solution = solution[left:-1][::-1]
    reverse_dist = []
    for offset in range(10):
        reverse_dist.append(np.cumsum(score(r_solution,offset)[::-1])[::-1])
    reverse_dist = np.array(reverse_dist)
    
    return reverse_dist

def calc_old_dist(solution,from_node):
    offset = from_node%10
    return np.cumsum(score(solution[from_node:],offset=offset))

def calc_middle(reverse_dist, offset):
    middle = np.zeros(shape=(reverse_dist.shape[1]))
    for o in range(10):
        order = (offset-o)%10
        using_reverse = reverse_dist[order]
        middle[o::10] = using_reverse[o::10]
    
    reverse_dist[:,:-1] -= reverse_dist[:,-1:]
    reverse_dist = reverse_dist[:,:-1]
    return middle[::-1], reverse_dist

def calc_connector(roller, left, node_before_a, node_a):
    to_d = roller[:-1]
    offset = left%10
    if node_before_a not in prime and offset == 0:
        to_d = to_d*1.1
    
    roller = distance(nodes[node_a],nodes[solution[left+2:]])
    if node_a not in prime:
        from_a = roller + roller*isten[left+1:]*0.1
    else:
        from_a = roller 
    
    return to_d, from_a, roller
        
#def print_on_iter(iter_num)

In [272]:
bests = []
dids = []

start = time.time()

for i in range(1,5):
    best = []
    did = 0
    
    isten = (np.arange(n)+1)%10==0
    reverse_dist = calc_reverse(1, solution)
    old_dist = calc_old_dist(solution,from_node=0)

    roller = distance(nodes[solution[0]],nodes[solution[0+2:]])


    for left in range(1,n-1):
        node_before_a, node_a = solution[left-1:left+1]
        offset = left%10

        middle, reverse_dist = calc_middle(reverse_dist, offset)

        to_d, from_a, roller =  calc_connector(roller, left, node_before_a, node_a)


        save = from_a+middle+to_d-old_dist[2:]
        old_dist[1:] -= old_dist[0]
        old_dist = old_dist[1:]

        place = np.argmin(save)
        saving = save[place]
        if saving<0:
            best.append(saving)
            did += saving

            target = np.argmin(save)+1+left
            solution[left:target+1] = solution[left:target+1][::-1]

            print('a: %s | d: %s | saving: %.3f | cumsave: %.3f'%(left, target, saving, did))
            reverse_dist = calc_reverse(left+1, solution)
            old_dist = calc_old_dist(solution,left)
            roller = distance(nodes[solution[left]],nodes[solution[left+2:]])

    pd.DataFrame({'Path':solution}).to_csv('./solution/iter_%s.csv'(%i+1),index=False)
    bests.append(best)
    dids.append(did)
    print('iter %s done, total swap %s, saves %.3f'%(i+1,len(best),did))
    print('####################################')

a: 1041 | d: 167717 | saving: -0.738 | cumsave: -0.738
a: 1778 | d: 1779 | saving: -0.524 | cumsave: -1.262
a: 8021 | d: 12731 | saving: -3.262 | cumsave: -4.524
a: 8022 | d: 12731 | saving: -6.717 | cumsave: -11.241
a: 8681 | d: 10199 | saving: -0.364 | cumsave: -11.605
a: 8763 | d: 10342 | saving: -1.999 | cumsave: -13.604
a: 14707 | d: 149066 | saving: -8.762 | cumsave: -22.366
a: 17001 | d: 21910 | saving: -3.304 | cumsave: -25.669
a: 17998 | d: 18827 | saving: -0.421 | cumsave: -26.090
a: 19310 | d: 47497 | saving: -6.297 | cumsave: -32.388
a: 19311 | d: 47497 | saving: -6.463 | cumsave: -38.851
a: 20718 | d: 21672 | saving: -0.009 | cumsave: -38.860
a: 25667 | d: 26344 | saving: -0.377 | cumsave: -39.237
a: 25962 | d: 28632 | saving: -3.525 | cumsave: -42.761
a: 26454 | d: 31189 | saving: -0.958 | cumsave: -43.720
a: 29721 | d: 137167 | saving: -0.050 | cumsave: -43.769
a: 31819 | d: 71484 | saving: -4.014 | cumsave: -47.783
a: 31828 | d: 71490 | saving: -3.867 | cumsave: -51.651

TypeError: must be str, not int

In [277]:
score(solution).sum()

1516777.3712323203

In [276]:
1516912.3751145517-score(solution).sum()

135.00388223142363

In [262]:
sum(best)

-1.2617374440491105

# UTILITY

In [20]:
#write result
pd.DataFrame({'Path':solution}).to_csv(path+'solution/submit2.csv',index=False)

In [21]:
#make .tsp problem
fname = path + 'problem/santa.tsp'
with open(fname, 'w') as f:
    f.write('NAME : santa\n')
    f.write('COMMENT : santa\n')
    f.write('TYPE : TSP\n')
    f.write('DIMENSION : %d\n' % n)
    f.write('EDGE_WEIGHT_TYPE : EUC_2D\n')
    f.write('NODE_COORD_SECTION\n')
    for idx,node in enumerate(nodes):
        x,y=node*100000
        f.write('%s %s %s\n' % (idx+1,x,y))
    f.write('EOF\n')

In [24]:
#read neos html
fname = path + 'solution/neos2.html'
l = 1
sol = []
with open(fname, 'r') as f:
    while l< 207274:
        line = f.readline()
        if l>=9505:
            sol.append(int(line.split(' ')[0]))
        l += 1
sol.append(0)
solution = np.array(sol)

# node swap

In [ ]:
start = time.time()

#every iter
coor = nodes[solution]
dist = score(solution)
neighbor = pd.DataFrame({'d':dist}).d.rolling(2,center=True).sum().values
oldnext = pd.DataFrame({'d':dist}).d.rolling(3,center=True).sum().values
notprime = ~np.isin(solution[:-1], prime)
isten = (np.arange(n)+1)%10==0
isextra = isten&notprime
better = 0

time_a = time.time()

cur = np.append([0],distance(coor[0],coor))
nex = np.append([0],distance(coor[1],coor))

swap = {}
for left in range(1,n-1):
#every left
    pre_coor, left_coor, next_coor = coor[left-1:left+2]
    adj_dist = distance(left_coor, next_coor)
    
    FROM_FLAG = (left+1)%10 == 0
    TO_FLAG = left%10 == 0
    LEFT_FLAG = notprime[left]
    LEFT_pre_FLAG = notprime[left-1]
    LEFT_nex_FLAG = notprime[left+1]
    
    ##old cost
    left_nb = neighbor[left]
    old_cost = neighbor[left:] + left_nb
    old_cost[0] = 0
    old_cost[1] = oldnext[left]

    ##new cost
    pre = cur[1:]
    
    cur = nex[1:]
    
    nex = distance(next_coor,coor[left-1:])

    cur_1 = cur[:-2]
    cur_2 = cur[2:]
    pre_ = pre[1:-1]
    nex_ = nex[1:-1]
    
    if TO_FLAG & LEFT_pre_FLAG:
        pre_ = pre_ + pre_*0.1
    if FROM_FLAG:
        nex_ = nex_ + nex_*notprime[left:]*0.1
        
    cur_1 = cur_1 + cur_1*isextra[left-1:-1]*0.1
    
    if LEFT_FLAG:
        cur_2 = cur_2 + cur_2*isten[left:]*0.1
    
    if FROM_FLAG & LEFT_nex_FLAG:
        adj_dist *= 1.1
    new_cost = pre_ + nex_ + cur_1 + cur_2
    
    
    new_cost[0] = 0
    new_cost[1] += adj_dist
    
    save = old_cost - new_cost
    good = np.where(save>0)[0]
    
    if len(good)>0:
        good_save = save[good]
        swap[left] = [good+left,good_save]
    
    if left%10000 == 0:
        time_b = time.time()
        print(left,time_b-time_a)
        time_a = time_b
print(time.time()-start)
print(len(swap))
###############################################
bk = set()
candidate = []
for left in swap:
    right,improve = swap[left]
    for i in range(len(right)):
        candidate.append((improve[i],right[i],left))
        
candidate = sorted(candidate)[::-1]
l=[]
r=[]
for improve,left,right in candidate:
    fill = [left-1,left,left+1,right-1,right,right+1]
    fill = [not (x in bk) for x in fill]
    if all(fill):
        l.append(left)
        r.append(right)
        bk.add(left-1)
        bk.add(left)
        bk.add(left+1)
        bk.add(right-1)
        bk.add(right)
        bk.add(right+1)

buf = solution[l]
solution[l] = solution[r]
solution[r] = buf

score(solution).sum()

10000 130.5129473209381
20000 119.38748025894165
